In [1]:
import pandas as pd

In [2]:
def remove_currency_signs(df, col):
    """Remove dollar sign and thousand separator."""
    df[col] = df[col].str.replace('$', '', regex=False)
    df[col] = df[col].str.replace(',', '')

In [3]:
def convert_to_bool(df, col):
    """Convert object columns to boolean."""
    df[col] = df[col].replace({'f': 0, 't': 1})
    df[col] = df[col].astype(bool)

## `calendar_cleaned`

In [4]:
calendar = pd.read_csv('data/Boston/calendar.csv')

In [5]:
calendar_cleaned = calendar.copy()

In [6]:
calendar_cleaned['date'] = pd.to_datetime(calendar_cleaned['date'])

convert_to_bool(calendar_cleaned, 'available')

remove_currency_signs(calendar_cleaned, 'price')
calendar_cleaned['price'] = calendar_cleaned['price'].astype(float)

In [7]:
calendar_cleaned.to_csv('data/Boston/cleaned_calendar.csv', index=False)

In [8]:
calendar_cleaned

,listing_id,date,available,price
0,12147973,2017-09-05,False,NaN
1,12147973,2017-09-04,False,NaN
2,12147973,2017-09-03,False,NaN
3,12147973,2017-09-02,False,NaN
4,12147973,2017-09-01,False,NaN
...,...,...,...,...
1308885,14504422,2016-09-10,False,NaN
1308886,14504422,2016-09-09,False,NaN
1308887,14504422,2016-09-08,False,NaN
1308888,14504422,2016-09-07,False,NaN


### `listings_cleaned`

In [9]:
listings = pd.read_csv('data/Boston/listings.csv')

In [10]:
listings.shape[0]

3585

In [11]:
listings_short = listings.copy()
drop_cols = ['listing_url', 'scrape_id', 'last_scraped', 'name', 'space', 'experiences_offered', 
             'neighborhood_overview', 'notes', 'transit', 'thumbnail_url', 'medium_url', 'picture_url', 
             'xl_picture_url', 'host_url', 'host_name', 'host_location', 'host_about', 'host_thumbnail_url', 
             'host_picture_url', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 
             'host_verifications', 'host_has_profile_pic', 'street', 'neighbourhood', 'neighbourhood_group_cleansed', 
             'city', 'state', 'zipcode', 'market', 'smart_location', 'country_code', 'country', 'latitude', 
             'longitude', 'is_location_exact', 'bathrooms', 'beds', 'bed_type', 'square_feet', 
             'guests_included', 'extra_people', 'maximum_nights', 'calendar_updated', 'has_availability', 
             'availability_30', 'availability_60', 'availability_90', 'availability_365', 
             'calendar_last_scraped', 'requires_license', 'license', 'jurisdiction_names', 
             'require_guest_profile_picture', 'require_guest_phone_verification', 'calculated_host_listings_count']
listings_short.drop(drop_cols, axis=1, inplace=True)

In [12]:
listings_cleaned = listings_short.copy()

In [13]:
listings_cleaned.drop(['summary', 'host_acceptance_rate'], axis=1, inplace=True)

In [14]:
index = listings_short[listings_short.property_type.isnull()].index
listings_short.loc[index, 'property_type'] = 'Other'

In [15]:
listings_cleaned['bedrooms'].fillna(1, inplace=True)
listings_cleaned['bedrooms'] = listings_cleaned['bedrooms'].astype(int)

In [16]:
index = listings_short[listings_short.bedrooms == 0].index
listings_short.loc[index, 'bedrooms'] = 1

In [17]:
listings_cleaned.drop(listings_cleaned[listings_cleaned.review_scores_accuracy.isnull()].index, inplace=True)
listings_cleaned.drop(listings_cleaned[listings_cleaned.review_scores_location.isnull()].index, inplace=True)
listings_cleaned.drop(listings_cleaned[listings_cleaned.review_scores_rating.isnull()].index, inplace=True)
listings_cleaned.drop(listings_cleaned[listings_cleaned.host_response_time.isnull()].index, inplace=True)

In [18]:
listings_cleaned.drop(['weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee',
                       'access', 'interaction', 'house_rules'], axis=1, inplace=True)

In [19]:
listings_cleaned.isnull().sum()

id                             0
description                    0
host_id                        0
host_since                     0
host_response_time             0
host_response_rate             0
host_is_superhost              0
host_identity_verified         0
neighbourhood_cleansed         0
property_type                  2
room_type                      0
accommodates                   0
bedrooms                       0
amenities                      0
price                          0
minimum_nights                 0
number_of_reviews              0
first_review                   0
last_review                    0
review_scores_rating           0
review_scores_accuracy         0
review_scores_cleanliness      0
review_scores_checkin          0
review_scores_communication    0
review_scores_location         0
review_scores_value            0
instant_bookable               0
cancellation_policy            0
reviews_per_month              0
dtype: int64

In [20]:
listings_cleaned.shape

(2541, 29)

**Summary:**

* *neighbourhood_cleansed* remains since it is the only one of the three without missing values
* missing values in *property_type* integrated in category *Other*
* replace NaN and 0 bedrooms with 1 
* drop Boston-specific columns *access*, *interaction* and *house_rules*
* drop columns *weekly_price*, *monthly_price*, *security_deposit* and *cleaning_fee* due to large amount of missing values
* drop rows with missing values for all host- and review-related columns

In [21]:
listings_cleaned.to_csv('data/Boston/cleaned_listings.csv', index=False)